# Distilbert experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import AutoModel, BertTokenizerFast,BertTokenizer, DistilBertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_5.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Distilbert

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

81


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.146194,0.127660,0.002240,0.017544,0.003972
2,No log,3.730649,0.269504,0.033112,0.052632,0.037215
3,No log,3.122725,0.382979,0.120257,0.169540,0.125296
4,No log,2.600642,0.446809,0.160783,0.224548,0.176723
5,No log,2.221478,0.510638,0.231396,0.301722,0.243151
6,No log,2.004055,0.517730,0.273049,0.309068,0.276373
7,3.154000,1.820278,0.567376,0.312049,0.371098,0.326460
8,3.154000,1.677085,0.595745,0.337791,0.414683,0.353926
9,3.154000,1.640173,0.567376,0.379391,0.404632,0.372411
10,3.154000,1.590184,0.553191,0.347744,0.362308,0.338835


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precis

TrainOutput(global_step=960, training_loss=2.0022032419840494, metrics={'train_runtime': 331.4228, 'train_samples_per_second': 45.875, 'train_steps_per_second': 2.897, 'total_flos': 2016871807168512.0, 'train_loss': 2.0022032419840494, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-1.5712891 , -1.5371094 , -2.4824219 , ..., -1.7304688 ,
        -1.3925781 ,  6.2539062 ],
       [-1.6494141 , -1.3554688 , -1.2910156 , ..., -1.2011719 ,
        -2.6367188 , -3.0078125 ],
       [-1.8935547 , -1.5566406 , -1.1484375 , ...,  0.9472656 ,
        -0.95996094, -1.1191406 ],
       ...,
       [-1.3349609 , -1.9560547 , -2.4648438 , ..., -1.4326172 ,
        -1.9160156 ,  2.7265625 ],
       [-1.8681641 , -2.8164062 , -1.3994141 , ..., -2.1621094 ,
        -1.1367188 , -2.34375   ],
       [-1.6054688 , -1.5566406 , -2.0566406 , ..., -1.7519531 ,
        -1.1279297 ,  7.5507812 ]], dtype=float32), label_ids=array([27, 31, 24, 50, 80,  1, 35, 80, 58, 80, 55, 57, 80, 44, 52, 66, 80,
       52,  6, 21, 55, 36, 50, 37, 29, 23, 77, 35, 70, 74, 35, 30, 53,  4,
       77, 52,  9, 61, 22, 80, 71, 56, 19, 17, 52, 51, 72, 11, 72, 74, 79,
       72,  3,  8, 53, 80,  8,  6, 72, 36,  9, 19, 77, 36,  6, 16, 70,  8,
       24, 65, 76, 80, 35, 80, 7

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.071974,0.184397,0.007805,0.034113,0.012555
2,No log,3.553566,0.276596,0.033160,0.065372,0.039923
3,No log,2.892026,0.347518,0.124457,0.149152,0.124966
4,No log,2.368693,0.453901,0.190790,0.228814,0.196380
5,No log,2.117875,0.482270,0.256173,0.283762,0.244708
6,No log,1.841966,0.539007,0.279522,0.318657,0.280121
7,2.896300,1.631534,0.574468,0.369034,0.385665,0.356304
8,2.896300,1.569408,0.602837,0.408557,0.443386,0.411801
9,2.896300,1.537603,0.609929,0.436980,0.450932,0.416026
10,2.896300,1.576677,0.602837,0.428776,0.427765,0.400200


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precis

TrainOutput(global_step=960, training_loss=1.6866068681081137, metrics={'train_runtime': 354.312, 'train_samples_per_second': 42.911, 'train_steps_per_second': 2.709, 'total_flos': 2016871807168512.0, 'train_loss': 1.6866068681081137, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-2.4511719 , -3.4414062 , -2.7089844 , ..., -2.0351562 ,
        -3.2011719 ,  7.6523438 ],
       [-4.5429688 , -3.6660156 , -1.8369141 , ..., -3.0175781 ,
        -2.6894531 , -3.8242188 ],
       [-4.2421875 , -3.4257812 , -3.7167969 , ..., -0.5415039 ,
        -4.5507812 , -2.8574219 ],
       ...,
       [-1.8330078 , -2.9882812 , -2.9863281 , ..., -1.6035156 ,
        -2.0234375 ,  2.4453125 ],
       [-4.3984375 , -2.9121094 , -0.35424805, ..., -2.5488281 ,
        -2.3476562 , -4.765625  ],
       [-3.8886719 , -4.4609375 , -2.0410156 , ..., -2.6582031 ,
        -3.7675781 ,  5.921875  ]], dtype=float32), label_ids=array([27, 31, 24, 50, 80,  1, 35, 80, 58, 80, 55, 57, 80, 44, 52, 66, 80,
       52,  6, 21, 55, 36, 50, 37, 29, 23, 77, 35, 70, 74, 35, 30, 53,  4,
       77, 52,  9, 61, 22, 80, 71, 56, 19, 17, 52, 51, 72, 11, 72, 74, 79,
       72,  3,  8, 53, 80,  8,  6, 72, 36,  9, 19, 77, 36,  6, 16, 70,  8,
       24, 65, 76, 80, 35, 80, 7